In [1]:
from __future__ import division

import time
import click
import os
import numpy as np
from scipy import stats


def calculate_cutie(label, samp_var1_fp, delimiter1, samp_var2_fp, delimiter2, 
                    f1type, f2type, working_dir, skip, startcol, endcol, 
                    statistic, resample_k, rm_zero, paired, alpha, mc, fold, 
                    fold_value, n_replicates,method):

    # calculate_cutie()
    start_time = time.clock()

    # file handling and parsing decisions
    # file 1 is the 'dominant' file type and should always contain the OTU file
    # we let the dominant fil 'override' the sample_id list ordering
    samp_ids, var2_names, samp_to_var2, n_var2, n_samp = \
        parse_input(f2type, samp_var2_fp, startcol, endcol, delimiter2, skip)
    samp_ids, var1_names, samp_to_var1, n_var1, n_samp = \
        parse_input(f1type, samp_var1_fp, startcol, endcol, delimiter1, skip)

    # temporary printing of samp and var names for reference
    print samp_ids[0:5]
    print var1_names[0:5]

    # convert dictionaries to matrices
    samp_var1, avg_var1, norm_avg_var1, var_var1, norm_var_var1, skew_var1 = \
        dict_to_matrix(samp_to_var1, samp_ids)
    samp_var2, avg_var2, norm_avg_var2, var_var2, norm_var_var2, skew_var2 = \
        dict_to_matrix(samp_to_var2, samp_ids)

    ###
    # Simple Linear Regression: Spearman and Pearson
    ### 
    # statistic-specific initial output
    stat_to_matrix = assign_statistics(samp_var1, samp_var2, 
                                                  statistic, rm_zero)

    # unpack statistic matrices
    pvalues = stat_to_matrix['pvalues']
    corrs = stat_to_matrix['correlations']
    logpvals = stat_to_matrix['logpvals']
    r2vals = stat_to_matrix['r2vals']

    # determine significance threshold and number of correlations
    threshold, n_corr = set_threshold(pvalues, alpha, mc, paired)

    # calculate initial sig candidates
    initial_sig, all_pairs = initial_sig_SLR(n_var1, n_var2, 
        pvalues, threshold, paired)

    # calculate true sig based on cutie resampling
    initial_insig = set(all_pairs).difference(initial_sig)

    # return sets of interest; some of these will be empty dicts depending on the statistic
    (true_sig, TP_comb_to_rev, 
     P_worst_p, P_worst_r, 
     false_insig, FN_comb_to_rev, 
     N_best_p, N_best_r) = updatek_cutie_SLR(initial_insig, 
            initial_sig, pvalues, samp_var1, samp_var2, threshold, 
            resample_k, corrs, fold, fold_value, working_dir, method,
            paired, statistic, n_replicates)

    print time.clock() - start_time
    return samp_ids, samp_var1, samp_var2, var1_names, var2_names, initial_sig, initial_insig, true_sig, \
        TP_comb_to_rev, P_worst_p, P_worst_r, \
        false_insig, FN_comb_to_rev, N_best_p, N_best_r

print 'Done'


Done


In [2]:
#!/usr/bin/env python
from __future__ import division

import re
import sys
import os
import csv
import numpy as np
from itertools import izip
from scipy import stats

def mapping_parse (samp_meta_file, startcol=17, endcol=100, delimiter='\t'):
    """
    """
    samp_ids = []
    samp_meta = {}
    # generate metabolite list from the 0th line (header)
    # default assumes metabolites are in col 17 to 99
    meta_names = samp_meta_file.readline().split(delimiter)[startcol:endcol]
    # for the remainder of the lines (i.e. the non-header lines)
    for line in samp_meta_file:
        if line != '\n':
            line = line.split('\n')[0]
            line = line.split(delimiter)
            samp_ids.append(line[0]) # line[0] is the sample id
            metabolite_levels = [np.nan if x == '' else float(x) for x in \
                line[startcol:endcol]]
            while len(metabolite_levels) < len(meta_names):
                metabolite_levels.append(np.nan)
            samp_meta[line[0]] = metabolite_levels
    n_meta = len(meta_names)
    n_samp = len(samp_ids)
    print 'The length of mapping_variables is ' + str(n_meta)
    print 'The number of samples is ' + str(n_samp)
    return samp_ids, meta_names, samp_meta, n_meta, n_samp


def otu_parse(samp_bact_file, delimiter = '\t', skip = 1):
    """
    """
    # create lists (corresponding to smoking and non-smoking files) 
    bact_names = []
    samp_bact = {}
    
    for i in xrange(skip):
        samp_bact_file.readline() # line 0 is 'constructed from biom file' 

    samp_ids = samp_bact_file.readline().rstrip().split(delimiter)
    samp_ids.pop(0) # the 0th entry is a header
    for samp_id in samp_ids:
        samp_bact[samp_id] = []

    for line in samp_bact_file:
        if line is not '': 
            split_line = line.rstrip().split(delimiter)
            # the 0th entry is the name of an OTU
            bact_names.append(split_line[0])
            split_line.pop(0) # pop off OTU
            for b in xrange(len(split_line)):
                samp_bact[samp_ids[b]].append(split_line[b])
        
    n_bact = len(bact_names)
    n_samp = len(samp_ids)

    print 'The length of samp_ids is ' + str(n_samp)
    print 'The length of bact_names is ' + str(n_bact)

    return samp_ids, bact_names, samp_bact, n_bact, n_samp

def parse_input(ftype, fp, startcol, endcol, delimiter, skip):
    """
    """
    # some files like the mapping file won't split on \n but will on \rU
    if ftype == 'map':
        with open(fp,'rU') as f:    
            samp_ids, var_names, samp_to_var, n_var, n_samp = \
                mapping_parse(f, startcol, endcol, delimiter)
   
    elif ftype == 'otu':
        with open(fp, 'rU') as f:
            samp_ids, var_names, samp_to_var, n_var, n_samp = \
                otu_parse(f, delimiter, skip)     

    return samp_ids, var_names, samp_to_var, n_var, n_samp 

def dict_to_matrix(samp_dict, samp_ids):
    """ 
    """
    
    # initialize matrix; # rows = # of samp_ids, # cols = # entries per key
    rows = len(samp_ids)
    cols = len(samp_dict[samp_dict.keys()[0]])
    samp_matrix = np.zeros(shape=(rows,cols))    
    
    # populate matrix from the dict
    for r in xrange(rows):
        for c in xrange(cols):
            samp_matrix[r][c] = samp_dict[samp_ids[r]][c]

    # retrieve mean values and normalize
    avg_matrix = np.array([np.nanmean(samp_matrix,0)])
    norm_avg_matrix = avg_matrix - avg_matrix.min()
    norm_avg_matrix = norm_avg_matrix/norm_avg_matrix.max()

    # retrieve variances and normalize
    var_matrix = np.array([np.nanvar(samp_matrix,0)])
    norm_var_matrix = var_matrix - var_matrix.min()
    norm_var_matrix = norm_var_matrix/var_matrix.max()

    skew_matrix = np.array([[stats.skew(samp_matrix[:,x],nan_policy='omit') \
        for x in xrange(cols)]])
    return samp_matrix, avg_matrix, norm_avg_matrix,var_matrix,norm_var_matrix, skew_matrix
    
def read_taxa(t):
    """
    """
    parts = t.split(';')
    while parts:
        if not parts[-1].endswith('__'):
            t1 = parts[-2].split('__')[1]
            t2 = parts[-1].split('__')[1]
            return t1 + ' ' + t2
        else:
            parts.pop()

    # This should not be reached: "k__;p__..."
    return 'Uncharacterized'

print 'Done'

Done


In [3]:
#!/usr/bin/env python
from __future__ import division
    
import os
import math
import itertools    
import numpy as np
from scipy import stats
try:
    import statsmodels.api as sm
except:
    pass

from collections import defaultdict

from cutie import parse
from cutie import output

def initial_stats_SLR(samp_var1, samp_var2, functions, mapf, f_stats, rm_zero = False):
    """ 
    INPUTS
    samp_var1: np array where row i col j corresponds to level of var1 j in 
               sample i
    samp_var2: np array where row i col j corresponds to level of var2 j in 
               sample i                  
    functions: list of strings of function names 
    mapf:      dict that maps function name to function object
    f_stats:   dict that maps function name to list of output strings
    rm_zero:   remove values that are 0 in both x and y

    OUTPUTS
    statistics: list of dict where each dict corresponds to each element 
                in function in the order they are passed and where each 
                key corresponds to a particular statistic calculated by 
                each function in functions and the element is an array 
                where row i col j corresponds to the value of a given 
                statistic to the values for var1 row i and var2 col j.
     
    FUNCTION
    Function that computes an initial set of statistics per the specified 
    functions. Returns a dict where the key is a statistical function and  
    the element is an initial matrix with dimensions n_rel_stats x n_var1 x 
    n_var2, corresponding to the relevant statistics from simple linear 
    regression (SLR) between each var1 and var2. 
    """
    n_var1, n_var2, n_samp = get_param(samp_var1, samp_var2)

    stat_dict = {}
    
    # retrieve relevant stats and create dictionary entry, 3D array
    for f in functions:
        rel_stats = f_stats[f]
        stat_dict[f] = np.zeros((len(rel_stats), 
                                 n_var1, 
                                 n_var2))

    # subset the data matrices into the cols needed
    for var1 in xrange(n_var1):
        for var2 in xrange(n_var2):
            var1_values = samp_var1[:,var1]
            var2_values = samp_var2[:,var2] 
            # remove zero values
            stacked = np.stack([var1_values,var2_values],0)
            if rm_zero is True:
                stacked = stacked[:,~np.all(stacked == 0.0, axis = 0)]
            # remove NANs
            stacked = stacked[:,np.all(~np.isnan(stacked), axis = 0)]
            var1_values = stacked[0]
            var2_values = stacked[1]
            for f in functions:
                # values is a list of the relevant_stats in order
                if len(var1_values) == 0 or len(var2_values) == 0: 
                    values = np.zeros([len(f_stats[f])])
                    values[:] = np.nan
                else:
                    values = mapf[f](var1_values, var2_values)
                for s in xrange(len(values)):
                    stat_dict[f][s][var1][var2] = values[s] 
    
    return stat_dict 

def assign_statistics(samp_var1, samp_var2, statistic, rm_zero = False):
    """
    samp_var1: np array where row i col j corresponds to level of var1 j in 
               sample i
    samp_var2: np array where row i col j corresponds to level of var2 j in 
               sample i        
    statistic: statistic of choice (e.g. kpc)
    rm_zero:   boolean whether zeros should be removed
    """
    n_var1, n_var2, n_samp = get_param(samp_var1, samp_var2)

    stat_to_matrix = {}

    if (statistic == 'kpc' or statistic == 'jkp' or statistic == 'rpc' or 
        statistic == 'bsp' or statistic == 'rjkp' or statistic == 'rbsp'):

        # define function and dictionary mapping string to function for statistic of interest
        functions = ['stats.linregress']
        mapf = {'stats.linregress': stats.linregress}
        f_stats = {'stats.linregress': 
                   ['b1', 'b0', 'pcorr','ppvalue','stderr']}
        stat_dict = initial_stats_SLR(samp_var1, samp_var2, functions, mapf,
                                      f_stats)
        
        stat_to_matrix['pvalues'] = stat_dict['stats.linregress'][3]
        stat_to_matrix['correlations'] = stat_dict['stats.linregress'][2]
        stat_to_matrix['logpvals'] = np.log(stat_dict['stats.linregress'][3])
        stat_to_matrix['r2vals'] = np.square(stat_dict['stats.linregress'][2])

    elif statistic == 'ksc':
        functions = ['stats.spearmanr']
        mapf = {'stats.spearmanr': stats.spearmanr}
        f_stats = {
        'stats.spearmanr': ['scorr','spvalue']}

        stat_dict = initial_stats_SLR(samp_var1, samp_var2, functions, mapf, 
                                      f_stats)
        
        stat_to_matrix['pvalues'] = stat_dict['stats.spearmanr'][1]
        stat_to_matrix['logpvals'] = np.log(stat_to_matrix['pvalues'])
        stat_to_matrix['correlations'] = stat_dict['stats.spearmanr'][0]
        stat_to_matrix['r2vals'] = [] # filler

    else:
        print 'Invalid statistic chosen'

    return stat_to_matrix

def set_threshold(pvalues, alpha, mc, paired = False):
    """
    INPUTS
    pvalues: 2D np array of pvalues
    alpha:   float of original cutoff
    mc:      form of multiple corrections to use
             nomc: none
             bc: bonferroni
             fwer: family-wise error rate 
             fdr: false discovery rate
    paired:  boolean, true if correlations are between a single matrix 

    OUTPUTS
    threshold: float cutoff of pvalues

    FUNCTION
    Performs a multiple comparisons correction on the alpha value
    """
    print 'The type of mc correction used was ' + mc
    pvalues_copy = np.copy(pvalues)
    if paired == True:
        # fill the upper diagonal with nan as to not double count pvalues in FDR
        pvalues_copy[np.triu_indices(pvalues_copy.shape[1],0)] = np.nan
        # currently computing all pairs double counting
        n_corr = np.size(pvalues_copy,1) * (np.size(pvalues_copy,1) - 1)#/2)
    else:
        n_corr = np.size(pvalues_copy,0) * np.size(pvalues_copy,1)

    # determine threshold based on multiple comparisons setting
    pvalues_copy = np.sort(pvalues_copy.flatten())
    pvalues_copy = pvalues_copy[~np.isnan(pvalues_copy)]
    if mc == 'nomc':
        threshold = alpha
    elif mc == 'bc':
        threshold = alpha / pvalues_copy.size
    elif mc == 'fwer':
        threshold = 1.0 - (1.0 - alpha) ** (1/(pvalues_copy.size))
    elif mc == 'fdr':
        # compute FDR cutoff
        cn = 1.0
        thresholds = np.array([(float(k+1))/(len(pvalues_copy))
            * alpha / cn for k in xrange(len(pvalues_copy))])
        compare = np.where(pvalues_copy <= thresholds)[0]
        if len(compare) is 0:
            threshold = alpha
            print 'Warning: no p-values below threshold, defaulted with min(p) = ' \
                + str(min(pvalues_copy))
        else:
            threshold = thresholds[max(compare)]
    print 'The threshold value was ' + str(threshold)
    return threshold, n_corr


def initialize_headers(infln_metrics):
    """
    Initialize headers for R matrix
    """
    headers = ['var1_index','var2_index','sample_number','var1_value', \
                'var2_value','initial_sig']

    for metric in infln_metrics:
        # populate headers
        headers.append(metric + '_indicator')
        headers.append(metric + '_cutoff')
        headers.append(metric + '_strength')

    return headers


def calculate_intersection(names, sets):
    '''
    Calculates intersection of items in sets for all regions
    e.g. 
    names = ['a','b','c']
    sets = [set_a, set_b, set_c] where set_i = set([1,2,3])
    '''
    # temporary mapping of name  to set
    name_to_set = {}
    for i in xrange(len(names)):
        name_to_set[names[i]] = sets[i]

    # get regions and initialize default dict of list
    regions = []
    for i in xrange(1, len(names)+1):
        els = [list(x) for x in itertools.combinations(names, i)]
        regions.extend(els)
    regions_set = defaultdict(list)

    # create union of sets
    union_set = set()
    for indiv_set in sets:
        union_set = union_set.union(indiv_set)

    # for each region, determine in_set and out_set
    for region in regions:
        in_set = set(region)
        out_set = set(names).difference(in_set)

        # for each in_set,     
        final_set = union_set
        for in_s in in_set:
            final_set = final_set.intersection(name_to_set[in_s])

        for out_s in out_set:
            final_set = final_set.difference(name_to_set[out_s])

        regions_set[str(region)] = final_set

        print 'The amount of unique elements in set ' + str(region) + ' is ' + str(len(final_set))
    
    return regions, regions_set



def get_param(samp_var1, samp_var2):
    """
    Extracts number of variables and samples
    """
    n_var1 = np.size(samp_var1, 1)
    n_var2 = np.size(samp_var2, 1)
    n_samp = np.size(samp_var1, 0)

    return n_var1, n_var2, n_samp


def initial_sig_SLR(n_var1, n_var2, pvalues, threshold, paired):
    """
    Determine list of initially significant candidate correlations
    """
    initial_sig = []
    all_pairs = []
    for var1 in xrange(n_var1): 
        for var2 in xrange(n_var2): 
            pair = (var1,var2)
            if not (paired and (var1 == var2)):
                all_pairs.append(pair)
            # if variables are paired i.e. the same, then don't compute corr(i,i)
            if pvalues[var1][var2] < threshold and not (paired and (var1 == var2)):
                initial_sig.append(pair)
    
    print 'The length of initial_sig is ' + str(len(initial_sig))
    print 'The length of initial_insig is ' + \
        str(len(set(all_pairs).difference(set(initial_sig))))
    return initial_sig, all_pairs


###
# RESAMPLE K
###


def updatek_cutie_SLR(initial_insig, initial_sig, pvalues, samp_var1, samp_var2, 
    threshold, resample_k, corrs, fold, fold_value, working_dir, method,
    paired = False, statistic = 'kpc', n_replicates = 1000):
    """
    Perform cutie resampling up to k points or other statistical analysis
    """
    n_var1, n_var2, n_samp = get_param(samp_var1, samp_var2)

    # raise error if resampling too many points
    if resample_k > n_samp - 3:
        raise ValueError('Too many points specified for resampling for size %s' 
            % (str(len(samp_ids))))
    
    (true_sig, TP_comb_to_rev, 
     P_worst_p, P_worst_r, 
     false_insig, FN_comb_to_rev, 
     N_best_p, N_best_r) = initialize_stat_dicts(resample_k, n_var1, n_var2)

    # separate FP and TP
    if (statistic == 'kpc' or statistic == 'ksc' or statistic == 'jkp' or 
        statistic == 'bsp'):
        (true_sig, TP_comb_to_rev, P_worst_p, P_worst_r, samp_counter, 
         var1_counter, var2_counter) = cutiek_true_sig(initial_sig, samp_var1,
            samp_var2, pvalues, corrs, threshold, paired, statistic, resample_k, 
            P_worst_p, P_worst_r, true_sig, TP_comb_to_rev, fold, fold_value, 
            n_replicates, method)

    # separate FN and TN
    if (statistic == 'rpc' or statistic == 'rsc' or statistic == 'rjkp' or 
        statistic == 'rbsp'): 
        (false_insig, FN_comb_to_rev, N_best_p, N_best_r, samp_counter, 
         var1_counter, var2_counter) = cutiek_false_insig(initial_insig, 
            samp_var1, samp_var2, pvalues, corrs, threshold, paired, statistic, 
            resample_k, N_best_p, N_best_r, false_insig, FN_comb_to_rev, fold, 
            fold_value, n_replicates, method)

    # output histograms/plots showing sample and variable appearance among CUtIe's
    output.diag_hist(samp_counter, var1_counter, var2_counter, resample_k, 
                     working_dir)

    return (true_sig, TP_comb_to_rev, P_worst_p, P_worst_r, false_insig, 
            FN_comb_to_rev, N_best_p, N_best_r)


def cutiek_true_sig(initial_sig, samp_var1, samp_var2, pvalues, corrs, 
    threshold, paired, statistic, resample_k, P_worst_p, P_worst_r, true_sig, 
    TP_comb_to_rev, fold, fold_value, n_replicates, method):
    """
    Determine true significant correlatoins via resampling of k points
    """     
    n_var1, n_var2, n_samp = get_param(samp_var1, samp_var2)
    # diagnostic statistics
    samp_counter = {}
    var1_counter = {}
    var2_counter = {}

    # initialize counter dictionaries for tracking sample and variable freq in CUtIe's
    for i in xrange(resample_k):
        samp_counter[str(i+1)] = np.zeros(n_samp)
        var1_counter[str(i+1)] = np.zeros(np.size(samp_var1,1))
        var2_counter[str(i+1)] = np.zeros(np.size(samp_var2,1)) 

    for pair in initial_sig:
        var1, var2 = pair
        # obtain sign of correlation
        sign = np.sign(corrs[var1][var2])

        insig = np.zeros(n_samp) # indicators for whether correlation is insig or not
        rev_corr = np.zeros(n_samp) # indicators for whether a sign reverses
        
        # resample_k = number of points being resampled
        for i in xrange(resample_k):
            new_rev_corr, new_insig, max_maxp, min_minr = evaluate_correlation_k(
                var1, var2, n_samp, samp_var1, samp_var2, pvalues, threshold, 
                statistic, i, sign, fold, fold_value, n_replicates, method) 

            # update the insig-indicators for the k-th resample iteration
            insig = np.add(insig, new_insig)
            rev_corr = np.add(rev_corr, new_rev_corr)

            # update the correlation within the resample_k dictionary
            P_worst_p[str(i+1)][var1][var2] = max_maxp
            P_worst_r[str(i+1)][var1][var2] = min_minr

            # sums to 0
            if insig.sum() == 0:
                true_sig[str(i+1)].append(pair)
                if rev_corr.sum() != 0:
                    TP_comb_to_rev[str(i+1)].append(pair)

            samp_counter[str(i+1)] = np.add(samp_counter[str(i+1)], insig)
            var1_counter[str(i+1)][var1] += 1
            var2_counter[str(i+1)][var2] += 1

    return (true_sig, TP_comb_to_rev, P_worst_p, P_worst_r, samp_counter, 
            var1_counter, var2_counter)



def cutiek_false_insig(initial_insig, samp_var1, samp_var2, pvalues, corrs, 
    threshold, paired, statistic, resample_k, N_best_p, N_best_r, false_insig, 
    FN_comb_to_rev, fold, fold_value, n_replicates, method):
    """
    Determine true significant correlatoins via resampling of k points
    """      
    n_var1, n_var2, n_samp = get_param(samp_var1, samp_var2)

    # diagnostic statistics
    samp_counter = {}
    var1_counter = {}
    var2_counter = {}

    # initialize counter dictionaries for tracking sample and variable freq in CUtIe's
    for i in xrange(resample_k):
        samp_counter[str(i+1)] = np.zeros(n_samp)
        var1_counter[str(i+1)] = np.zeros(np.size(samp_var1,1))
        var2_counter[str(i+1)] = np.zeros(np.size(samp_var2,1)) 

    for pair in initial_insig:
        var1, var2 = pair
        # obtain sign of correlation
        sign = np.sign(corrs[var1][var2])

        insig = np.zeros(n_samp) # indicators for whether correlation is insig or not
        rev_corr = np.zeros(n_samp) # indicators for whether a sign reverses
        
        # resample_k = number of points being resampled
        for i in xrange(resample_k):
            new_rev_corr, new_insig, min_minp, max_maxr = evaluate_correlation_k(
                var1, var2, n_samp, samp_var1, samp_var2, pvalues, threshold, 
                statistic, i, sign, fold, fold_value, n_replicates, method) 

            # update the insig-indicators for the k-th resample iteration
            insig = np.add(insig, new_insig)
            rev_corr = np.add(rev_corr, new_rev_corr)

            # update the correlation within the resample_k dictionary
            N_best_p[str(i+1)][var1][var2] = min_minp
            N_best_r[str(i+1)][var1][var2] = max_maxr

            # sums to 0
            if insig.sum() != 0:
                false_insig[str(i+1)].append(pair)
                if rev_corr.sum() != 0:
                    FN_comb_to_rev[str(i+1)].append(pair)

            samp_counter[str(i+1)] = np.add(samp_counter[str(i+1)], insig)
            var1_counter[str(i+1)][var1] += 1
            var2_counter[str(i+1)][var2] += 1

    return (false_insig, FN_comb_to_rev, N_best_p, N_best_r, samp_counter, 
            var1_counter, var2_counter)



def evaluate_correlation_k(var1, var2, n_samp, samp_var1, samp_var2, pvalues, 
    threshold, statistic, i, sign, fold, fold_value, n_replicates, method):
    """
    Evaluate a given var1, var2 correlation at the resample_k = i level 
    """

    if statistic == 'kpc':
        new_rev_corr, new_insig, maxp, minr = resamplek_cutie_pc(var1, var2, 
            n_samp, samp_var1, samp_var2, pvalues, threshold, i + 1, sign, fold, 
            fold_value)

    elif statistic == 'rpc':
        new_rev_corr, new_insig, minp, maxr = reversek_cutie_pc(var1, var2, 
            n_samp, samp_var1, samp_var2, pvalues, threshold, i + 1, sign, fold,
            fold_value)

    elif statistic == 'ksc':
        new_rev_corr, new_insig, maxp, minr = resamplek_cutie_sc(var1, var2, 
            n_samp, samp_var1, samp_var2, pvalues, threshold, i + 1, sign, fold, 
            fold_value)

    elif statistic == 'jkp':
        new_rev_corr, new_insig, maxp, minr = jackknifek_cutie_pc(var1, var2, 
            n_samp, samp_var1, samp_var2, pvalues, threshold, i + 1, sign, method)

    elif statistic == 'bsp':
        new_rev_corr, new_insig, maxp, minr = bootstrap_cutie_pc(var1, var2, 
            n_samp, samp_var1, samp_var2, pvalues, threshold, sign, method,
            n_replicates)

    elif statistic == 'rjkp':
        new_rev_corr, new_insig, minp, maxr= jackknifek_revcutie_pc(var1, var2, 
            n_samp, samp_var1, samp_var2, pvalues, threshold, i + 1, sign, method)

    elif statistic == 'rbsp':
        new_rev_corr, new_insig, minp, maxr = bootstrap_revcutie_pc(var1, var2, 
            n_samp, samp_var1, samp_var2, pvalues, threshold, sign, method,
            n_replicates)

    else:
        raise ValueError('Invalid statistic chosen %s' % statistic)
    
    # obtain most extreme p and R-sq values
    if (statistic == 'kpc' or statistic == 'ksc' or statistic == 'jkp' or 
        statistic == 'bsp'):
        extrema_p = np.max(maxp)
        extrema_r = np.min(minr)

    elif (statistic == 'rpc' or statistic == 'rsc' or statistic == 'rjkp' or 
          statistic == 'rbsp'):
        extrema_p = np.min(minp)
        extrema_r = np.max(maxr)

    return new_rev_corr, new_insig, extrema_p, extrema_r


def compute_pc(new_var1, new_var2):
    """
    compute pearson correlation and return p and r values
    """

    # if resulting variables do not contain enough points
    if new_var1.size < 2 or new_var2.size < 2:
        p_value = 1
        r_value = 0

    else:
        slope, intercept, r_value, p_value, std_err = stats.linregress(
                                                        new_var1, new_var2)
    # if p_value is nan
    if np.isnan(p_value):
        p_value = 1
        r_value = 0

    return p_value, r_value


def update_rev_maxp_minr(sign, r_value, p_value, indices, reverse, maxp, minr):
    """
    Check sign, r and p value and update reverse, maxp, and minr
    """
    # if sign has reversed
    if np.sign(r_value) != sign:
        for i in indices:
            reverse[i] += 1

    # update most extreme p and r values
    for i in indices:
        if p_value > maxp[i]:
            maxp[i] = p_value
        if np.absolute(r_value) < np.absolute(minr[i]):
            minr[i] = r_value

    return reverse, maxp, minr


def update_revrev_minp_maxr(sign, r_value, p_value, indices, reverse, minp, maxr):
    """
    Check sign, r and p value and update reverse, maxp, and minr
    """
    # if sign has reversed
    if np.sign(r_value) != sign:
        for i in indices:
            reverse[i] += 1

    # update most extreme p and r values
    for i in indices:
        if p_value < minp[i]:
            minp[i] = p_value
        if np.absolute(r_value) > np.absolute(maxr[i]):
            maxr[i] = r_value

    return reverse, minp, maxr

def init_var_indicators(var1_index, var2_index, samp_var1, samp_var2):
    """
    Initialize indicator matrices and variable matrices
    """
    n_var1, n_var2, n_samp = get_param(samp_var1, samp_var2)

    exceeds = np.zeros(n_samp)
    reverse = np.zeros(n_samp)
    maxp = np.zeros(n_samp)
    minr = np.ones(n_samp)

    # slice relevant variables
    var1 = samp_var1[:,var1_index]
    var2 = samp_var2[:,var2_index]
    return exceeds, reverse, maxp, minr, var1, var2 


def init_rev_var_indicators(var1_index, var2_index, samp_var1, samp_var2):
    """
    Initialize indicator matrices and variable matrices
    """
    n_var1, n_var2, n_samp = get_param(samp_var1, samp_var2)

    exceeds = np.zeros(n_samp)
    reverse = np.zeros(n_samp)
    minp = np.ones(n_samp)
    maxr = np.zeros(n_samp)

    # slice relevant variables
    var1 = samp_var1[:,var1_index]
    var2 = samp_var2[:,var2_index]
    return exceeds, reverse, minp, maxr, var1, var2 

def remove_nans(var1, var2):
    """
    Remove Nan Points
    # remove all points where one or both values are NAN
    # new_var1 = np.array([1,2,np.nan])
    # new_var2 = np.array([1,np.nan,3])
    # stacked = array([[  1.,   2.,  nan],
    #                  [  1.,  nan,   3.]])
    # np.isnan(stacked) = array([[False, False,  True],
    #                             [False,  True, False]], dtype=bool)
    # np.all(~np.isnan(stacked), axis = 0) = array([ True, False, False], dtype=bool)
    # stacked[:,np.all(~np.isnan(stacked), axis = 0)] =  array([[ 1.],
    #                                                           [ 1.]])
    """
    stacked = np.stack([var1, var2], 0)
    stacked = stacked[:,np.all(~np.isnan(stacked), axis = 0)]
    new_var1 = stacked[0]
    new_var2 = stacked[1]
    return new_var1, new_var2

def jackknifek_cutie_pc(var1_index, var2_index, n_samp, samp_var1, samp_var2,
                       pvalues, threshold, resample_k, sign, method):
    """
    Perform jackknife resampling using Pearson
    """
    # initialize indicators and variables
    exceeds, reverse, maxp, minr, var1, var2 = init_var_indicators(var1_index,
                                            var2_index, samp_var1, samp_var2)
    #corr_nan = False
    p_values = []
    # iteratively delete k samples and recompute statistics
    combs = [list(x) for x in itertools.combinations(xrange(n_samp), resample_k)]
    for indices in combs:
        new_var1 = var1[~np.in1d(range(len(var1)), indices)]
        new_var2 = var2[~np.in1d(range(len(var2)), indices)]

        # remove NaNs
        new_var1, new_var2 = remove_nans(new_var1, new_var2)

        # compute new p_value and r_value
        p_value, r_value = compute_pc(new_var1, new_var2)

        # if p-value is nan; break loop and set exceeds += 1

        # update reverse, maxp, and minr
        reverse, maxp, minr = update_rev_maxp_minr(sign, r_value, p_value,
                                                   indices, reverse, maxp, minr)

        p_values.append(p_value)


    # generate log confidence interval on p-value
    CI, p_mu, p_sigma = get_pCI(np.array(p_values), n_samp, method)

    # test confidence interval
    exceeds = test_upper_CI(CI, threshold, exceeds, [item for sublist in combs for item in sublist], method)

    return reverse, exceeds, maxp, minr


def jackknifek_revcutie_pc(var1_index, var2_index, n_samp, samp_var1, samp_var2,
                       pvalues, threshold, resample_k, sign, method):
    """
    Perform reverse jackknife resampling using Pearson
    """
    # initialize indicators and variables
    exceeds, reverse, minp, maxr, var1, var2 = init_rev_var_indicators(var1_index,
                                            var2_index, samp_var1, samp_var2)

    p_values = []
    # iteratively delete k samples and recompute statistics
    combs = [list(x) for x in itertools.combinations(xrange(n_samp), resample_k)]
    for indices in combs:
        new_var1 = var1[~np.in1d(range(len(var1)), indices)]
        new_var2 = var2[~np.in1d(range(len(var2)), indices)]

        # remove NaNs
        new_var1, new_var2 = remove_nans(new_var1, new_var2)

        # compute new p_value and r_value
        p_value, r_value = compute_pc(new_var1, new_var2)


        # update reverse, maxp, and minr
        reverse, maxp, minr = update_revrev_minp_maxr(sign, r_value, p_value,
                                                   indices, reverse, minp, maxr)

        p_values.append(p_value)

    # generate log confidence interval on p-value
    CI, p_mu, p_sigma = get_pCI(np.array(p_values), n_samp, method)

    # test confidence interval
    # https://stackoverflow.com/questions/952914/making-a-flat-list-out-of-list-of-lists-in-python    
    exceeds = test_lower_CI(CI, threshold, exceeds, [item for sublist in combs for item in sublist], method)

    return reverse, exceeds, minp, maxr

def bootstrap_cutie_pc(var1_index, var2_index, n_samp, samp_var1, samp_var2,
                       pvalues, threshold, sign, method, n_replicates = 1000):
    """
    Bootstrap resampling, n_replicates and log P confidence interval
    """
    # initialize indicators and variables
    exceeds, reverse, maxp, minr, var1, var2 = init_var_indicators(var1_index,
                                            var2_index, samp_var1, samp_var2)

    p_values = []

    for k in xrange(n_replicates):
        new_samp = np.random.choice(xrange(n_samp), size=n_samp, replace=True)
        new_var1 = []
        new_var2 = []
        for j in xrange(n_samp):
            new_var1.append(var1[new_samp[j]])
            new_var2.append(var2[new_samp[j]])
    

        # remove NaNs
        new_var1, new_var2 = remove_nans(new_var1, new_var2)

        # compute new p_value and r_value
        p_value, r_value = compute_pc(new_var1, new_var2)


        # update reverse, maxp, and minr
        reverse, maxp, minr = update_rev_maxp_minr(sign, r_value, p_value,
                                                   xrange(n_samp), reverse, maxp, minr)
        
        p_values.append(p_value)

    CI, p_mu, p_sigma = get_pCI(np.array(p_values), n_samp, method)

    exceeds = test_upper_CI(CI, threshold, exceeds, xrange(n_samp), method)
       
    #maxp = [CI[1]] * n_samp 


    return reverse, exceeds, maxp, minr
   


def bootstrap_revcutie_pc(var1_index, var2_index, n_samp, samp_var1, samp_var2,
                       pvalues, threshold, sign, method, n_replicates = 1000):
    """
    Bootstrap resampling, n_replicates and log P confidence interval
    """
    # initialize indicators and variables
    exceeds, reverse, minp, maxr, var1, var2 = init_rev_var_indicators(var1_index,
                                            var2_index, samp_var1, samp_var2)

    p_values = []

    for k in xrange(n_replicates):
        new_samp = np.random.choice(xrange(n_samp), size=n_samp, replace=True)
        new_var1 = []
        new_var2 = []
        for j in xrange(n_samp):
            new_var1.append(var1[new_samp[j]])
            new_var2.append(var2[new_samp[j]])
    

        # remove NaNs
        new_var1, new_var2 = remove_nans(new_var1, new_var2)

        # compute new p_value and r_value
        p_value, r_value = compute_pc(new_var1, new_var2)


        # update reverse, maxp, and minr
        reverse, minp, maxr = update_revrev_minp_maxr(sign, r_value, p_value,
                                                   xrange(n_samp), reverse, minp, maxr)
        
        p_values.append(p_value)

    CI, p_mu, p_sigma = get_pCI(np.array(p_values), n_samp, method)

    exceeds = test_lower_CI(CI, threshold, exceeds, xrange(n_samp), method)
       
    #maxp = [CI[1]] * n_samp 

    return reverse, exceeds, minp, maxr
   
def reversek_cutie_pc(var1_index, var2_index, n_samp, samp_var1, samp_var2,
                       pvalues, threshold, resample_k, sign, fold, fold_value):
    """
    Perform cutie to detect FN
    """
    # initialize indicators and variables
    # exceeds, reverse, maxp, minr, var1, var2 
    exceeds, reverse, maxr, minp, var1, var2 = init_var_indicators(var1_index,
                                            var2_index, samp_var1, samp_var2)

    # iteratively delete two samples and recompute statistics
    combs = [list(x) for x in itertools.combinations(xrange(n_samp), resample_k)]
    for indices in combs:
        new_var1 = var1[~np.in1d(range(len(var1)),indices)]
        new_var2 = var2[~np.in1d(range(len(var2)),indices)]
        # remove NaNs
        new_var1, new_var2 = remove_nans(new_var1, new_var2)

        # compute new p_value and r_value
        p_value, r_value = compute_pc(new_var1, new_var2)

        if fold:
            if (p_value < threshold and \
                p_value < pvalues[var1_index][var2_index] * fold_value) or \
                np.isnan(p_value):
                for i in indices:
                    exceeds[i] += 1
        elif p_value < threshold or np.isnan(p_value): 
            for i in indices:
                exceeds[i] += 1 # exceeds is a good thing here!


        if np.sign(r_value) != sign:
            for i in indices:
                reverse[i] += 1

        for i in indices:
            if p_value < minp[i]:
                minp[i] = p_value
            if r_value > np.absolute(maxr[i]):
                maxr[i] = r_value

    return reverse, exceeds, minp, maxr

def resamplek_cutie_pc(var1_index, var2_index, n_samp, samp_var1, samp_var2,
                       pvalues, threshold, resample_k, sign, fold, fold_value):
    """     
    INPUTS
    var1_index: integer of var1 (in var1_names) to be evaluated
    var2_index: integer of var1 (in var2_names) to be evaluated
    n_samp:     integer of number of samples
    samp_var1:  np array where row i col j indicates level of bact j 
                for sample i
    samp_var2:  np array where row i col j indicates level of meta j 
                for sample i
    pvalues:    np array of pvalues
    threshold:  float of level of significance testing (after MC)
    sign:       original sign of correlation to check against following re-evaluation
    fold:       boolean if using 100x criterion for identifying cuties

    OUTPUTS
    reverse: array where index i is 1 if the correlation changes sign upon removing
             sample i 
    exceeds: array where index i is x if removing that sample causes the 
             correlation to become insignificant in x different pairwise correlations
    maxp:    maximum p-value observed via resampling k points
    minr:    minimum p-value observed via resampling k points 

    FUNCTION
    Takes a given bacteria and metabolite by index and recomputes pearson correlation 
    by removing 1 out of n (sample_size) points from samp_ids. 
    Returns an indicator array where exceeds[i] is 1 if removing that sample causes
    the correlation to become insignificant in k different pairwise correlations
    """
    # initialize indicators and variables
    exceeds, reverse, maxp, minr, var1, var2 = init_var_indicators(var1_index,
                                            var2_index, samp_var1, samp_var2)

    # iteratively delete k samples and recompute statistics
    combs = [list(x) for x in itertools.combinations(xrange(n_samp), resample_k)]
    for indices in combs:
        new_var1 = var1[~np.in1d(range(len(var1)),indices)]
        new_var2 = var2[~np.in1d(range(len(var2)),indices)]
       
        # remove NaNs
        new_var1, new_var2 = remove_nans(new_var1, new_var2)

        # compute new p_value and r_value
        p_value, r_value = compute_pc(new_var1, new_var2)


        # update reverse, maxp, and minr
        reverse, maxp, minr = update_rev_maxp_minr(sign, r_value, p_value,
                                                   indices, reverse, maxp, minr)
        
        if fold:
            if (p_value > threshold and \
                p_value > pvalues[var1_index][var2_index] * fold_value) or \
                np.isnan(p_value):
                for i in indices:
                    exceeds[i] += 1
        elif p_value > threshold or np.isnan(p_value): 
            for i in indices:
                exceeds[i] += 1

    return reverse, exceeds, maxp, minr
    
def resamplek_cutie_sc(var1_index, var2_index, n_samp, samp_var1, 
    samp_var2, pvalues, threshold, k, sign, fold, fold_value, rm_zero = False):
    """     
    INPUTS
    rm_zero:          remove values that are 0 in both x and y
        
    OUTPUTS
    reverse: array where index i is 1 if the correlation changes sign upon removing
             sample i 
    exceeds: array where index i is k if removing that sample causes the 
             correlation to become insignificant in k different pairwise correlations
    
    FUNCTION
    Takes a given bacteria and metabolite by index and recomputes spearman correlation 
    by removing 1 out of n (sample_size) points from samp_ids. 
    Returns an indicator array where exceeds[i] is 1 if removing that sample causes
    the correlation to become insignificant in k different pairwise correlations
    """
    # initialize indicators and variables
    exceeds, reverse, maxp, minr, var1, var2 = init_var_indicators(var1_index,
                                            var2_index, samp_var1, samp_var2)

    # iteratively delete two samples and recompute statistics
    combs = [list(x) for x in itertools.combinations(xrange(n_samp), k)]
    for indices in combs:
        new_var1 = var1[~np.in1d(range(len(var1)),indices)]
        new_var2 = var2[~np.in1d(range(len(var2)),indices)]
        stacked = np.stack([new_var1,new_var2],0)
        if rm_zero is True:
            stacked = stacked[:,~np.all(stacked == 0.0, axis = 0)]
        stacked = stacked[:,np.all(~np.isnan(stacked), axis = 0)]
        new_var1 = stacked[0]
        new_var2 = stacked[1]


        if new_var1.size <= 3 or new_var2.size <= 3:
            p_value = 1
            corr = 0
        else:
            corr, p_value = stats.spearmanr(new_var1, new_var2)
        if fold:
            if (p_value > threshold and p_value > pvalues[var1_index][var2_index] * fold_value) or np.isnan(p_value): # or np.sign(corr) != sign:
                for i in indices:
                    exceeds[i] += 1
        elif p_value > threshold or np.isnan(p_value): # or np.sign(corr) != sign:
            for i in indices:
                    exceeds[i] += 1
        if np.sign(corr) != sign:
            for i in indices:
                reverse[i] += 1
        
        for i in indices:
            if p_value > maxp[i]:
                maxp[i] = p_value
            if corr < np.absolute(minr[i]):
                minr[i] = corr

    return reverse, exceeds, maxp, minr

def get_pCI(p_values, n_samp, method = 'log', zero_replace = 10e-100):
    """
    Compute logp confidence interval
    """
    method = str(method)
    if method == 'log':
        p_values[p_values == 0] = zero_replace
        logp_values = np.log(p_values)
        p_mu = np.mean(logp_values)
        p_sigma = np.std(logp_values)
    elif method == 'cbrt':
        cbrtp_values = np.cbrt(p_values)
        p_mu = np.mean(cbrtp_values)
        p_sigma = np.std(cbrtp_values)
    elif method == 'none':
        p_mu = np.mean(p_values)
        p_sigma = np.std(p_values)

    pCI = (p_mu - 1.96 * p_sigma / np.sqrt(n_samp), p_mu + 1.96 * p_sigma / np.sqrt(n_samp))

    return pCI, p_mu, p_sigma

def test_upper_CI(CI, threshold, exceeds, indices, method = 'log'):
    """
    Test if upper bound of CI is above a threshold and update exceeds indicator matrix
    """

    method = str(method)
    if method == 'log':
        if CI[1] > np.log(threshold):
            for i in indices:
                exceeds[i] += 1
    elif method == 'cbrt':
        if CI[1] > np.cbrt(threshold):
            for i in indices:
                exceeds[i] += 1
    elif method == 'none':
        if CI[1] > threshold:
            for i in indices:
                exceeds[i] += 1

    return exceeds 


def test_lower_CI(CI, threshold, exceeds, indices, method = 'log'):
    """
    Test if lower bound of CI is below a threshold and update exceeds indicator matrix
    """
    method = str(method)
    if method == 'log':
        if CI[0] < np.log(threshold):
            for i in indices:
                exceeds[i] += 1
    elif method == 'cbrt':
        if CI[0] < np.cbrt(threshold):
            for i in indices:
                exceeds[i] += 1
    elif method == 'none':
        if CI[0] < threshold:
            for i in indices:
                exceeds[i] += 1
    return exceeds 

def initialize_stat_dicts(resample_k, n_var1, n_var2):
    """
    Create empty dicts
    """
    # create dicts of points to track true_sig and reversed-sign correlations
    true_sig = {}
    false_insig = {}
    FN_comb_to_rev = {}
    TP_comb_to_rev = {} 

    # create matrices dict to hold the most extreme values of p and r (for R-sq)
    P_worst_p = {}
    P_worst_r = {}
    N_best_p = {}
    N_best_r = {}

    # initialize dictionary entries as empty lists
    for i in xrange(resample_k):
        true_sig[str(i+1)] = []
        TP_comb_to_rev[str(i+1)] = []
        P_worst_p[str(i+1)] = np.zeros([n_var1, n_var2])
        P_worst_r[str(i+1)] = np.ones([n_var1, n_var2])
        
        false_insig[str(i+1)] = []
        FN_comb_to_rev[str(i+1)] = []
        N_best_p[str(i+1)] = np.ones([n_var1, n_var2])
        N_best_r[str(i+1)] = np.zeros([n_var1, n_var2])

    return (true_sig, TP_comb_to_rev, P_worst_p, P_worst_r, false_insig, 
            FN_comb_to_rev, N_best_p, N_best_r)

print 'Done'

Done


In [4]:
label = 'L6'
samp_var1_fp = 'data/simulated_data/input_tables/ts_1/txts/table_2.txt'
samp_var2_fp = 'data/simulated_data/input_tables/ts_1/txts/table_2.txt'
f1type = 'otu'
f2type = 'otu' 
working_dir = 'data_analysis/sim_ts3_kpc1fdr0.05/'
skip = 1
statistic = 'kpc'
resample_k = 1
paired = True
alpha = 0.05
mc = 'fdr'

# defaults
delimiter1 = '\t'
delimiter2 = '\t'
startcol = 17
endcol = 100
rm_zero = False
fold = True
fold_value = 1
n_replicates = 100
method = 'none'

samp_ids, samp_var1, samp_var2, var1_names, var2_names, initial_sig, initial_insig, true_sig, TP_comb_to_rev, P_worst_p, P_worst_r, \
        placeholder1, placeholder2, placeholder3, placeholder4 = calculate_cutie(label, samp_var1_fp, delimiter1, 
                                                                          samp_var2_fp, delimiter2, 
                    f1type, f2type, working_dir, skip, startcol, endcol, 
                    statistic, resample_k, rm_zero, paired, alpha, mc, fold, 
                    fold_value, n_replicates,method)

print 'Done'

The length of samp_ids is 50
The length of bact_names is 500
The length of samp_ids is 50
The length of bact_names is 500
['s0', 's1', 's2', 's3', 's4']
['o0', 'o1', 'o2', 'o3', 'o4']
The type of mc correction used was fdr
The threshold value was 0.00133306613226
The length of initial_sig is 6652
The length of initial_insig is 242848
144.573884
Done


/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:108: RuntimeWarning: divide by zero encountered in log


In [11]:
label = 'L6'
samp_var1_fp = 'data/simulated_data/input_tables/ts_1/txts/table_2.txt'
samp_var2_fp = 'data/simulated_data/input_tables/ts_1/txts/table_2.txt'
f1type = 'otu'
f2type = 'otu' 
working_dir = 'data_analysis/sim_ts3_kpc1fdr0.05/'
skip = 1
statistic = 'rpc'
resample_k = 1
paired = True
alpha = 0.05
mc = 'fdr'

# defaults
delimiter1 = '\t'
delimiter2 = '\t'
startcol = 17
endcol = 100
rm_zero = False
fold = True
fold_value = 1
n_replicates = 100
method = 'none'

samp_ids, samp_var1, samp_var2, var1_names, var2_names, initial_sig, initial_insig, placeholder5, placeholder6, placeholder7, placeholder8, \
        false_insig, FN_comb_to_rev, N_best_p, N_best_r = calculate_cutie(label, samp_var1_fp, delimiter1, 
                                                                          samp_var2_fp, delimiter2, 
                    f1type, f2type, working_dir, skip, startcol, endcol, 
                    statistic, resample_k, rm_zero, paired, alpha, mc, fold, 
                    fold_value, n_replicates,method)

print 'Done'

The length of samp_ids is 50
The length of bact_names is 500
The length of samp_ids is 50
The length of bact_names is 500
['s0', 's1', 's2', 's3', 's4']
['o0', 'o1', 'o2', 'o3', 'o4']
The type of mc correction used was fdr
The threshold value was 0.00133306613226
The length of initial_sig is 6652
The length of initial_insig is 242848
4044.19012
Done


/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:108: RuntimeWarning: divide by zero encountered in log


In [13]:
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split



import pandas as pd
df = pd.read_csv(samp_var1_fp, sep="\t", header=0, skiprows=1)
df = df.set_index('#OTU ID').T
#data.columns = ["a", "b", "c", "etc."]

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# input from script: mine_fp = otu_table_small.MSQ34_L6.csv, label = 'otu'
# obtain MINE output for full dataset 
# results stored in otu_table_small.MSQ34_L6.csv,allpairs,cv=0.1,B=n^0.6,Results.csv
# parameters are default, will be changed if needed

# sed script to delete one column at a time


#if mine_non_par == True:
# subset data files  
# PP package, forkfun
parse.subset_data(n_samp, transposed_fn, transposed_fp, working_dir)

def subset_data(n_samp, transposed_fn, transposed_fp, working_dir):
    for k in xrange(n_samp): 
        resample_fp = working_dir + str(k) + '_' + transposed_fn
        if os.path.isfile(resample_fp) == False:
            # sed to delete row
            # sed is 1 indexed, the top row is the header, hence the k + 2
            os.system("sed " + str(k+2)+ "d " + transposed_fp + " > " + \
                resample_fp)
    return

# run MINE on each subset
statistics.mine_subsets(n_samp, transposed_fn, transposed_fp, \
                        original_mine_fp, working_dir, label)

def mine_subsets(n_samp, transposed_fn, transposed_fp, original_mine_fp, working_dir, label):
    # resample_fp = working_dir/0_otu_transpose_table_small.MSQ34_L6.csv
    for k in xrange(n_samp): 
        resample_fp = working_dir + str(k) + '_' + transposed_fn
        # check if MINE results file for subset exists
        # example original_mine_fp:
        # otu_table_small.MSQ34_L6.csv,allpairs,cv=0.1,B=n^0.6,Results.csv
        # original_mine_fp.split(label)[1] = \
        # _table_small.MSQ34_L6.csv,allpairs,cv=0.1,B=n^0.6,Results.csv
        # subset_output_fp = working_dir/ + 0 + _ + otu + \
        # _table_small.MSQ34_L6.csv,allpairs,cv=0.1,B=n^0.6,Results.csv
        subset_output_fp = working_dir + str(k) + '_' + label + \
            original_mine_fp.split(label)[1]

        if os.path.isfile(subset_output_fp) == False:
            os.system("java -jar ../MINE/MINE.jar " + resample_fp + \
                " '-allPairs' cv=0.1 exp=0.6 c=10 fewBoxes")

return 

# declare initial stats to parsex
# provide statistic headers as headers in MINE output are less wieldy
mine_stats = ['MIC_str','MIC_nonlin','MAS_nonmono','MEV_func',
              'MCN_comp','linear_corr'] 
with open(original_mine_fp, 'rU') as f:
    stat_to_matrix = parse.parse_mine(f, n_var1, var1_names, 
                                        mine_stats, mine_delimiter)

with open(minep_fp, 'rU') as f:
    mine_bins, pvalues_ordered = parse.parse_minep(f, mine_delimiter,
                                                     pskip)

#print mine_bins
#print pvalues_ordered
# Store MINE_str and compute pvalue for each correlation 
mine_str = stat_to_matrix['MIC_str']
mine_nonlin = stat_to_matrix['MIC_nonlin']
mine_pvalues = 

# Exhaustively parse all subsetted files and store results in 3d arrays
mine_subset_str, mine_subset_p, mine_subset_nonlin = statistics.subset_mine(
                        n_samp, n_var1, var1_names, label, mine_stats, 
                        pvalues_ordered, mine_bins, working_dir, 
                        original_mine_fp, mine_delimiter)

# set FDR threshold
threshold, n_corr = statistics.set_threshold(mine_pvalues, alpha, mc, 
                                                paired)

# split into initial sig and true sig
initial_sig, true_sig = statistics.cutie_mine(n_samp, n_var1, threshold, 
                                            mine_pvalues, mine_subset_p)
### initial statistics       
print 'Length of initial sig MINE is ' + str(len(initial_sig))
print 'Length of true sig MINE is ' + str(len(true_sig['1']))


# obtain worst_mine_p value and worst_mine_str for each 
worst_mine_p = np.amax(mine_subset_p, axis=0)
worst_mine_str = np.amin(mine_subset_str, axis=0)
max_mine_subset_nonlin = np.amax(mine_subset_nonlin, axis=0)
min_mine_subset_nonlin = np.amin(mine_subset_nonlin, axis=0)

# empty dictionary as placeholder
P_comb_to_rev = {}


In [1]:
import pandas as pd


In [16]:
df = pd.read_csv("~/Desktop/Clemente Lab/CUtIe/data/simulated_data/input_tables/ts_1/txts_cutie/copula_table1_n50_lognorm_3_0.txt", 
                 delimiter = '\t', 
                 skiprows = 1)
df = df.set_index(list(df)[0])
df.head()


,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49
#OTU ID,,,,,,,,,,,,,,,,,,,,,
o0,0.009537,0.052656,0.272251,0.719604,63.118556,5.116637,0.028943,0.113538,0.134295,0.150017,...,0.044790,0.332401,5.440243,0.899963,3.689245,0.527316,8.582046,866.065909,2.522821,2.143449
o1,0.011433,0.044868,0.264934,0.787321,66.574168,3.986228,0.015542,0.166735,0.155948,0.124077,...,0.073157,0.383805,6.434144,1.389110,4.776380,0.391505,4.278387,749.632993,2.038918,1.988746
o2,0.013954,0.034109,0.234459,0.592649,71.007963,4.993145,0.013265,0.143507,0.175888,0.102469,...,0.055852,0.363486,5.975985,1.462398,4.145146,0.352022,4.223221,544.593745,2.079512,1.868052
o3,0.013664,0.034721,0.208439,0.548785,66.471393,6.740144,0.021424,0.193507,0.196872,0.098215,...,0.055309,0.277147,4.721589,1.426179,3.206538,0.423588,3.425147,556.090300,2.457044,2.133624
o4,0.012807,0.048692,0.198182,0.916384,65.927064,6.836099,0.015092,0.262787,0.222090,0.155372,...,0.069016,0.284375,3.377818,1.678714,3.279644,0.580139,3.218535,692.225700,2.126584,2.872972


In [34]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import LocalOutlierFactor

In [60]:
np.random.seed(42)

# fit the model
clf = LocalOutlierFactor(n_neighbors=20)
y_pred = clf.fit_predict(df.T)
out_df = pd.DataFrame(data = y_pred, index = range(len(list(df)))).reset_index()
out_df.to_csv('test.txt', sep = '\t', header = ['sample','outlier'])

In [56]:
print out_df

     0
0   -1
1    1
2    1
3    1
4    1
5   -1
6    1
7    1
8    1
9    1
10   1
11  -1
12   1
13   1
14   1
15   1
16   1
17   1
18   1
19   1
20   1
21   1
22   1
23  -1
24   1
25   1
26   1
27   1
28   1
29   1
..  ..
130  1
131  1
132  1
133  1
134  1
135  1
136  1
137  1
138  1
139  1
140  1
141  1
142  1
143 -1
144  1
145 -1
146  1
147  1
148 -1
149  1
150  1
151  1
152 -1
153  1
154  1
155 -1
156  1
157  1
158  1
159  1

[160 rows x 1 columns]


In [5]:
df = pd.read_csv("~/Desktop/Clemente Lab/CUtIe/data/pre_sparcc_MSQ/otu_table.MSQ34_L6.txt", 
                 delimiter = '\t', 
                 skiprows = 1)
df = df.set_index(list(df)[0])
df.head()


,37.LLL.58.65,53.LLL.64,66.RM.65,62.LLL.65,54.RM.64,70.LLL.65,68.Buc.65,51.RLL.64,51.Buc.64,45.LLL.64,...,770193.RM.65,770203.1736.B.65,770195.RM.65,770171.Brush.64,770192.LLL.65,770195.LLL.65,31.RM.58.65,770177.RM.65,770188.LLL.65,33.RM.58.65
#OTU ID,,,,,,,,,,,,,,,,,,,,,
k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;o__Cenarchaeales;f__Cenarchaeaceae;g__,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;o__Cenarchaeales;f__Cenarchaeaceae;g__Nitrosopumilus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;o__Cenarchaeales;f__SAGMA-X;g__,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
k__Archaea;p__Crenarchaeota;c__Thaumarchaeota;o__Nitrososphaerales;f__Nitrososphaeraceae;g__Candidatus Nitrososphaera,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
k__Archaea;p__Euryarchaeota;c__Halobacteria;o__Halobacteriales;f__Halobacteriaceae;g__Haloarcula,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
import pandas as pd
df = pd.read_csv("/Users/KevinBu/Desktop/Clemente Lab/CUtIe/data/simulated_data/input_tables/ts_1/txts_cutie/copula_table1_n50_lognorm_3_0_indep.txt", 
                 delimiter = '\t', 
                 skiprows = 1)
df = df.set_index(list(df)[0])
df.head()


,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49
#OTU ID,,,,,,,,,,,,,,,,,,,,,
o0,198.771721,3.152505,5.300961,1.040519,0.010064,0.007758,119.083014,2.759216,6.296837,24.684921,...,2.691605,6.120082,0.051975,0.078959,0.092821,0.623511,1.586171,0.015250,0.361753,0.037288
o1,3.321683,0.902373,14.547536,0.693617,0.005882,5.572708,5.507811,76.736404,252.421404,0.495792,...,0.998561,6.977364,0.006615,0.272016,0.057573,0.026255,0.002019,0.097206,113.208039,23.398924
o2,18.844365,26.817121,0.281691,2.765379,1.148441,301.224127,0.709311,0.525458,2.255277,0.736817,...,11.638839,0.542997,34.513298,96.122554,1.770100,8.301042,0.004804,0.001146,0.120505,764.570210
o3,831.041387,0.495272,1.369084,0.170521,0.056409,0.097589,2.127379,146.806717,30.245363,0.639396,...,3.613371,4.198004,209.392156,0.472891,0.138500,0.084360,0.065644,0.027368,0.020933,0.404060
o4,271.150487,0.352624,1.982106,0.068054,0.784715,0.568776,0.026448,0.077726,0.005434,8.119664,...,0.000547,12.508647,1.098576,0.027195,2.347198,16.698330,2.672506,22.992043,2.639005,2.675739


In [50]:
df = df.T
df = df.dropna(how = 'any', axis = 0)
#df = df.set_index(list(df)[0])
df.head()

#OTU ID,o0,o1,o2,o3,o4,o5,o6,o7,o8,o9,...,o490,o491,o492,o493,o494,o495,o496,o497,o498,o499
s0,198.771721,3.321683,18.844365,831.041387,271.150487,0.053299,17.292368,0.635037,0.733699,3.427378,...,0.045511,0.349997,27.135776,49.110159,3257.359438,0.801096,0.138670,0.213803,0.047164,0.791707
s1,3.152505,0.902373,26.817121,0.495272,0.352624,0.174854,0.007462,0.009065,0.029087,49.614554,...,3.755039,1.709804,0.090875,2.059293,2.380621,3.450822,0.551454,1.326544,0.031974,0.341522
s2,5.300961,14.547536,0.281691,1.369084,1.982106,1.830227,5.064831,0.004278,0.862455,2.048486,...,0.049959,0.108565,0.183325,4.170743,0.001543,52.229792,0.487780,0.476932,0.039241,0.709868
s3,1.040519,0.693617,2.765379,0.170521,0.068054,5.180929,1.344473,1.806774,23.976682,0.046528,...,56.140980,0.108630,51.697685,0.378942,1.810285,1.340781,66.991802,1.608499,0.032526,0.019587
s4,0.010064,0.005882,1.148441,0.056409,0.784715,0.121047,0.099028,0.236328,8.254485,16.239320,...,1.525667,4.139044,261.318149,69.893222,0.403234,19.473184,5.798228,30.239960,7.499600,0.053799


In [68]:
import pandas as pd
df = pd.read_csv("/Users/KevinBu/Desktop/Clemente Lab/CUtIe/data/MINE/WHOfix.txt", 
                 delimiter = '\t', 
                 skiprows = 0)
df = df.set_index(list(df)[0])
 
n_samp = 202
startcol = 3
endcol = 357
df = df.iloc[:,(startcol-1):(endcol-(startcol-1))]
df = df.dropna(how = 'any', axis = 0)
np.random.seed(42)
# fit the model
clf = LocalOutlierFactor(n_neighbors=20)
try:
    y_pred = clf.fit_predict(df)
except:
    y_pred = np.ones(n_samp)

#.shape[0] gives number of rows
out_df = pd.DataFrame(data = y_pred, index = range(n_samp)).reset_index()

In [48]:
df = df.T
#df = df.dropna(how = 'any', axis = 0)
#df = df.set_index(list(df)[0])
df.head()

Country,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United States of America,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
Adolescent fertility rate (%),151.0,27.0,6.0,NaN,146.0,NaN,62.0,30.0,16.0,14.0,...,43.0,64.0,40.0,92.0,81.0,25.0,NaN,83.0,161.0,101.0
Adult literacy rate (%),28.0,98.7,69.9,NaN,67.4,NaN,97.2,99.4,NaN,NaN,...,NaN,96.8,NaN,75.5,93.0,90.3,NaN,54.1,68.0,89.5
Gross national income per capita (PPP international $),NaN,6000.0,5940.0,NaN,3890.0,15130.0,11670.0,4950.0,33940.0,36040.0,...,44070.0,9940.0,2190.0,3480.0,10970.0,2310.0,NaN,2090.0,1140.0,NaN
Net primary school enrolment ratio female (%),NaN,93.0,94.0,83.0,49.0,NaN,98.0,84.0,97.0,98.0,...,93.0,100.0,78.0,86.0,91.0,91.0,NaN,65.0,94.0,88.0
Net primary school enrolment ratio male (%),NaN,94.0,96.0,83.0,51.0,NaN,99.0,80.0,96.0,97.0,...,91.0,100.0,79.0,88.0,91.0,96.0,NaN,85.0,90.0,87.0


In [44]:
import numpy as np
print len(df.columns)
np.random.seed(42)
#df.T.head()
# fit the model
clf = LocalOutlierFactor(n_neighbors=20)
y_pred = clf.fit_predict(df.T)
print np.array(range(len(y_pred)))[y_pred == -1]


353


ValueError: Found array with 0 feature(s) (shape=(353, 0)) while a minimum of 1 is required.

In [11]:
import numpy as np
print np.mean((df == 0).astype(int).sum(axis=1).values)/160


0.895955882353


In [35]:
import scipy.stats


In [68]:
# 58, 255
x = df.ix[45]
y = df.ix[21]
print x.value_counts()
print y.value_counts()
print scipy.stats.pearsonr(x,y)
print scipy.stats.pearsonr(x.drop('51.RLL.64'),y.drop('51.RLL.64'))
print scipy.stats.spearmanr(x,y)
print scipy.stats.spearmanr(x.drop('131016.N.RL.64'),y.drop('131016.N.RL.64'))



0.000000    159
0.000073      1
Name: k__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Cellulomonadaceae;g__Demequina, dtype: int64
0.000000    154
0.002527      1
0.003282      1
0.007964      1
0.000165      1
0.000254      1
0.000067      1
Name: k__Bacteria;p__Acidobacteria;c__Solibacteres;o__Solibacterales;f__;g__, dtype: int64
(0.018567692401223926, 0.81572772373866953)
(0.01850589323428636, 0.81690379921529366)
SpearmanrResult(correlation=0.39907460407713852, pvalue=1.7211038181258791e-07)
SpearmanrResult(correlation=nan, pvalue=nan)


//anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3162: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
//anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3163: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
//anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
//anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
//anaconda/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [66]:
x

37.LLL.58.65          0.000000
53.LLL.64             0.000000
66.RM.65              0.000000
62.LLL.65             0.000000
54.RM.64              0.000000
70.LLL.65             0.000000
68.Buc.65             0.000000
51.RLL.64             0.000000
51.Buc.64             0.000000
45.LLL.64             0.000000
55.LLL.64             0.000000
38.RM.64              0.000000
50.BUC.64             0.000000
68.LLL.65             0.000000
770179.RM.65          0.000000
770190.B.65           0.000000
131016.N.RL.64        0.000073
770195.B.65           0.000000
770171.RM.64          0.000000
57.RM.64              0.000000
770187.B.65           0.000000
110502BC.N.1.RL.64    0.000000
770186.LLL.65         0.000000
770172.LLL.64         0.000000
770172.RM.64          0.000000
770181.B.65           0.000000
101018WG.N.1.RL.64    0.000000
68.RML.65             0.000000
50.RM.64              0.000000
770180.B.65           0.000000
                        ...   
770179.B.65           0.000000
770192.B

In [62]:
y.value_counts()

0.000000    159
0.004326      1
Name: k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__Sporolactobacillaceae;g__, dtype: int64

In [2]:
import pandas as pd
df = pd.read_csv('~/Desktop/Clemente Lab/CUtIe/data/MINE/WHOfix.txt', 
                 delimiter = '\t', 
                 skiprows = 0)
df = df.set_index(list(df)[0])
df.head()

,CountryID,Continent,Adolescent fertility rate (%),Adult literacy rate (%),Gross national income per capita (PPP international $),Net primary school enrolment ratio female (%),Net primary school enrolment ratio male (%),Population (in thousands) total,Population annual growth rate (%),Population in urban areas (%),...,Total_CO2_emissions,Total_income,Total_reserves,Trade_balance_goods_and_services,Under_five_mortality_from_CME,Under_five_mortality_from_IHME,Under_five_mortality_rate,Urban_population,Urban_population_growth,Urban_population_pct_of_total
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1,1,151.0,28.0,NaN,NaN,NaN,26088.0,4.0,23.0,...,692.50,NaN,NaN,NaN,257.00,231.9,257.00,5740436.0,5.44,22.9
Albania,2,2,27.0,98.7,6000.0,93.0,94.0,3172.0,0.6,46.0,...,3499.12,4.790000e+09,78.14,-2.040000e+09,18.47,15.5,18.47,1431793.9,2.21,45.4
Algeria,3,3,6.0,69.9,5940.0,94.0,96.0,33351.0,1.5,64.0,...,137535.56,6.970000e+10,351.36,4.700000e+09,40.00,31.2,40.00,20800000.0,2.61,63.3
Andorra,4,2,NaN,NaN,NaN,83.0,83.0,74.0,1.0,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,5,3,146.0,67.4,3890.0,49.0,51.0,16557.0,2.8,54.0,...,8991.46,1.490000e+10,27.13,9.140000e+09,164.10,242.5,164.10,8578749.0,4.14,53.3


In [11]:
df.index.values[36]

'China'

In [12]:
list(df)[168]

'Aid_given'

In [103]:
# 58, 255
print list(df)[326], list(df)[303]

x = df.ix[:,326]
y = df.ix[:,303]
#print x.value_counts()
#print y.value_counts()
stacked = np.stack([x,y],0)
stacked = stacked[:,np.all(~np.isnan(stacked), axis = 0)]
x = stacked[0]
y = stacked[1]
print scipy.stats.pearsonr(x,y)
#print scipy.stats.pearsonr(x.drop('51.RLL.64'),y.drop('51.RLL.64'))
print scipy.stats.spearmanr(x,y)
#print scipy.stats.spearmanr(x.drop('131016.N.RL.64'),y.drop('131016.N.RL.64'))

print x
print y

Prostate_cancer_new_cases_per_100_000_men Oil_consumption
(0.29329927004084355, 0.01866603875487579)
SpearmanrResult(correlation=0.017788868797445723, pvalue=0.88904153579114742)
[   5.6    36.8    76.     71.4     7.5     0.3    19.     74.2    53.2
   14.8    78.2    40.6     1.7    48.3    38.1    39.3    35.9     4.4
   84.4    59.3    60.5    26.2    10.46   34.     81.      4.6     7.
    5.4    56.3    40.5    12.6     9.8     7.6     9.6    32.3     8.7
   29.9    56.7   100.9    81.8     5.6    46.     18.6    24.1    46.8
    9.5    16.8    12.8     8.5    13.8    30.5    42.9    35.9    90.9
   77.3     4.5     8.      2.5    14.5    10.4    52.2   124.8     2.4
   42.4 ]
[   251.35    421.42    885.7     294.37    108.19     95.72    142.86
    815.43   2047.53    109.22   2247.32    256.31   6984.12    224.62
    211.15    194.97    167.68    628.64    232.76   1960.21   2604.8
    434.29    285.17    163.41     20.82   2569.04   1231.8    1577.94
    195.61   1819.03   53

In [85]:
df = pd.read_csv('~/Desktop/Clemente Lab/CUtIe/data/lung_pt/Mapping.Pneumotype.Multiomics.RL.NYU.w_metabolites.w_inflamm.txt', 
                 delimiter = '\t', 
                 skiprows = 0)
df = df.set_index(list(df)[0])
df = df.iloc[:,16:99]
df.head()


,glutamic_acid,glycine,alanine,succinic_acid,aspartic_acid,glutamine,serine,methionine,urea,sucrose,...,myristic_acid,1_5_anhydroglucitol,azelaic_acid,behenic_acid,palmitoleic_acid,dihydroabietic_acid,pentadecanoic_acid,threitol,octadecanol,1_monostearin
#SampleID,,,,,,,,,,,,,,,,,,,,,
100716FG.C.1.RL,60088.12140,8.110014e+05,144276.0444,30070.95780,67135.16160,386941.680600,48898.92780,27381.24780,6.677151e+06,6939.451800,...,3.999599e+05,86877.6330,47231.30760,26197.77540,131688.20160,203772.429600,4.797905e+05,5218.037400,36795.232800,42228.447000
100804MB.C.1.RL,120075.13440,1.931722e+06,197367.0879,84091.07970,98779.75800,950017.529700,44450.89110,18537.24030,8.246570e+06,21936.803400,...,7.797187e+05,330270.7623,43232.17980,73379.24880,273568.61550,677860.053600,9.592541e+05,18408.954900,127579.830300,91339.204800
100907LG.C.1.RL,19712.98000,1.569294e+05,51957.7830,213252.20150,38933.13550,693544.878500,28231.80350,22810.73400,1.427332e+07,17108.050500,...,5.918822e+05,648345.8315,126867.10700,57449.25600,281191.57900,548020.844000,8.000654e+05,23373.962000,47874.380000,76528.604500
101007PC.C.1.RL,17490.17800,1.268707e+06,90846.0422,297384.46770,42182.19400,241261.573000,26338.15040,47480.68910,1.038325e+07,10288.340000,...,1.184599e+06,910055.1147,8282.11370,22891.55650,305100.72270,286273.060500,1.412126e+06,13940.700700,26852.567400,47223.480600
101018WG.N.1.RL,73103.15772,1.937921e+05,254635.0241,61839.64963,23902.55961,9848.093782,26992.54855,12718.79319,7.705834e+06,7356.167218,...,2.947451e+05,151010.7498,2930.50564,10047.44791,13755.43464,9050.677283,4.363862e+04,2950.441053,3289.343065,9848.093782


In [86]:
cols = list(df)
for col in cols:
    col_zscore = col + '_zscore'
    df[col_zscore] = (df[col] - df[col].mean())/df[col].std(ddof=0)

In [89]:
df

,glutamic_acid,glycine,alanine,succinic_acid,aspartic_acid,glutamine,serine,methionine,urea,sucrose,...,myristic_acid_zscore,1_5_anhydroglucitol_zscore,azelaic_acid_zscore,behenic_acid_zscore,palmitoleic_acid_zscore,dihydroabietic_acid_zscore,pentadecanoic_acid_zscore,threitol_zscore,octadecanol_zscore,1_monostearin_zscore
#SampleID,,,,,,,,,,,,,,,,,,,,,
100716FG.C.1.RL,60088.12140,8.110014e+05,1.442760e+05,30070.95780,67135.161600,3.869417e+05,48898.927800,27381.247800,6.677151e+06,6939.451800,...,-0.154014,-0.490959,0.023406,-0.178554,-0.252197,-0.026283,-0.064912,-0.464823,-0.268445,-0.170103
100804MB.C.1.RL,120075.13440,1.931722e+06,1.973671e+05,84091.07970,98779.758000,9.500175e+05,44450.891100,18537.240300,8.246570e+06,21936.803400,...,0.448364,-0.347697,-0.035368,1.121355,0.545285,1.958865,0.587249,-0.159595,0.287325,0.139540
100907LG.C.1.RL,19712.98000,1.569294e+05,5.195778e+04,213252.20150,38933.135500,6.935449e+05,28231.803500,22810.734000,1.427332e+07,17108.050500,...,0.150416,-0.160478,1.193778,0.682464,0.588133,1.415189,0.370722,-0.044708,-0.200620,0.046160
101007PC.C.1.RL,17490.17800,1.268707e+06,9.084604e+04,297384.46770,42182.194000,2.412616e+05,26338.150400,47480.689100,1.038325e+07,10288.340000,...,1.090590,-0.006435,-0.549014,-0.269645,0.722521,0.319172,1.203240,-0.262987,-0.329312,-0.138609
101018WG.N.1.RL,73103.15772,1.937921e+05,2.546350e+05,61839.64963,23902.559610,9.848094e+03,26992.548550,12718.793190,7.705834e+06,7356.167218,...,-0.320907,-0.453210,-0.627664,-0.623516,-0.915074,-0.841642,-0.658160,-0.517293,-0.473563,-0.374261
101019AB.N.1.RL,91490.47715,4.593119e+04,1.996344e+05,24608.21100,12913.628780,1.070282e+04,8822.591180,12169.803760,1.086499e+07,3347.212579,...,-0.541941,-0.345710,-0.607268,-0.577566,-0.927354,-0.849692,-0.677862,-0.502375,-0.473461,-0.347637
101026RM.C.1.RL,74320.14650,1.801788e+06,2.080495e+05,130272.64150,17723.172500,1.711092e+05,70248.211000,30436.985500,1.143933e+07,21179.923500,...,-0.386353,-0.145149,0.775844,-0.415269,-0.425141,-0.223896,-0.302042,0.072631,-0.357762,-0.254606
101109JD.N.1.RL,223226.12320,1.154628e+05,5.118831e+05,33441.78939,66883.578790,2.004764e+04,33558.007590,20454.404630,6.536228e+06,3225.055275,...,-0.085521,-0.408327,-0.607109,-0.432850,-0.876277,-0.827834,-0.594057,-0.498165,-0.463106,-0.324058
101206DM.N.1.RL,65152.37963,4.499920e+04,2.859891e+05,12792.41389,14365.812390,9.864525e+03,13025.003230,14803.627630,5.183513e+06,1778.624392,...,-0.538448,-0.403655,-0.657260,-0.643634,-0.917564,-0.851296,-0.682382,-0.531112,-0.069719,-0.420998


In [13]:
df.head()

,CountryID,Continent,Adolescent fertility rate (%),Adult literacy rate (%),Gross national income per capita (PPP international $),Net primary school enrolment ratio female (%),Net primary school enrolment ratio male (%),Population (in thousands) total,Population annual growth rate (%),Population in urban areas (%),...,Total_CO2_emissions,Total_income,Total_reserves,Trade_balance_goods_and_services,Under_five_mortality_from_CME,Under_five_mortality_from_IHME,Under_five_mortality_rate,Urban_population,Urban_population_growth,Urban_population_pct_of_total
Country,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1,1,151.0,28.0,NaN,NaN,NaN,26088.0,4.0,23.0,...,692.50,NaN,NaN,NaN,257.00,231.9,257.00,5740436.0,5.44,22.9
Albania,2,2,27.0,98.7,6000.0,93.0,94.0,3172.0,0.6,46.0,...,3499.12,4.790000e+09,78.14,-2.040000e+09,18.47,15.5,18.47,1431793.9,2.21,45.4
Algeria,3,3,6.0,69.9,5940.0,94.0,96.0,33351.0,1.5,64.0,...,137535.56,6.970000e+10,351.36,4.700000e+09,40.00,31.2,40.00,20800000.0,2.61,63.3
Andorra,4,2,NaN,NaN,NaN,83.0,83.0,74.0,1.0,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Angola,5,3,146.0,67.4,3890.0,49.0,51.0,16557.0,2.8,54.0,...,8991.46,1.490000e+10,27.13,9.140000e+09,164.10,242.5,164.10,8578749.0,4.14,53.3


In [17]:
import numpy as np
import minepy

def print_stats(mine):
    print "MIC", mine.mic()
    print "MAS", mine.mas()
    print "MEV", mine.mev()
    print "MCN (eps=0)", mine.mcn(0)
    print "MCN (eps=1-MIC)", mine.mcn_general()
    print "GMIC", mine.gmic()
    print "TIC", mine.tic()

x = np.linspace(0, 1, 1000)
y = np.sin(10 * np.pi * x) + x
mine = MINE(alpha=0.6, c=15, est="mic_approx")
mine.compute_score(x, y)

print "Without noise:"
print_stats(mine)
print

np.random.seed(0)
y +=np.random.uniform(-1, 1, x.shape[0]) # add some noise
mine.compute_score(x, y)

print "With noise:"
print_stats(mine)

Without noise:
MIC 1.0
MAS 0.726071574374
MEV 1.0
MCN (eps=0) 4.58496250072
MCN (eps=1-MIC) 4.58496250072
GMIC 0.779360251901
TIC 67.6612295532

With noise:
MIC 0.505716693417
MAS 0.365399904262
MEV 0.505716693417
MCN (eps=0) 5.95419631039
MCN (eps=1-MIC) 3.80735492206
GMIC 0.359475501353
TIC 28.7498326953


NameError: name 'hi' is not defined

In [5]:
import pandas as pd
df = pd.read_csv("/Users/KevinBu/Desktop/Clemente Lab/CUtIe/data/simulated_data/input_tables/ts_1/txts_cutie/copula_table1_n50_lognorm_3_0_indep.txt", 
                 delimiter = '\t', 
                 skiprows = 1)
df = df.set_index(list(df)[0])
#df = df.T
#df = df.dropna(how = 'any', axis = 0)
#df = df.set_index(list(df)[0])
df.head()


,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49
#OTU ID,,,,,,,,,,,,,,,,,,,,,
o0,198.771721,3.152505,5.300961,1.040519,0.010064,0.007758,119.083014,2.759216,6.296837,24.684921,...,2.691605,6.120082,0.051975,0.078959,0.092821,0.623511,1.586171,0.015250,0.361753,0.037288
o1,3.321683,0.902373,14.547536,0.693617,0.005882,5.572708,5.507811,76.736404,252.421404,0.495792,...,0.998561,6.977364,0.006615,0.272016,0.057573,0.026255,0.002019,0.097206,113.208039,23.398924
o2,18.844365,26.817121,0.281691,2.765379,1.148441,301.224127,0.709311,0.525458,2.255277,0.736817,...,11.638839,0.542997,34.513298,96.122554,1.770100,8.301042,0.004804,0.001146,0.120505,764.570210
o3,831.041387,0.495272,1.369084,0.170521,0.056409,0.097589,2.127379,146.806717,30.245363,0.639396,...,3.613371,4.198004,209.392156,0.472891,0.138500,0.084360,0.065644,0.027368,0.020933,0.404060
o4,271.150487,0.352624,1.982106,0.068054,0.784715,0.568776,0.026448,0.077726,0.005434,8.119664,...,0.000547,12.508647,1.098576,0.027195,2.347198,16.698330,2.672506,22.992043,2.639005,2.675739


In [10]:
import minepy
test = minepy.pstats(df, alpha=0.6, c=15, est="mic_approx")

In [7]:
df.head()

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s40,s41,s42,s43,s44,s45,s46,s47,s48,s49
#OTU ID,,,,,,,,,,,,,,,,,,,,,
o0,198.771721,3.152505,5.300961,1.040519,0.010064,0.007758,119.083014,2.759216,6.296837,24.684921,...,2.691605,6.120082,0.051975,0.078959,0.092821,0.623511,1.586171,0.015250,0.361753,0.037288
o1,3.321683,0.902373,14.547536,0.693617,0.005882,5.572708,5.507811,76.736404,252.421404,0.495792,...,0.998561,6.977364,0.006615,0.272016,0.057573,0.026255,0.002019,0.097206,113.208039,23.398924
o2,18.844365,26.817121,0.281691,2.765379,1.148441,301.224127,0.709311,0.525458,2.255277,0.736817,...,11.638839,0.542997,34.513298,96.122554,1.770100,8.301042,0.004804,0.001146,0.120505,764.570210
o3,831.041387,0.495272,1.369084,0.170521,0.056409,0.097589,2.127379,146.806717,30.245363,0.639396,...,3.613371,4.198004,209.392156,0.472891,0.138500,0.084360,0.065644,0.027368,0.020933,0.404060
o4,271.150487,0.352624,1.982106,0.068054,0.784715,0.568776,0.026448,0.077726,0.005434,8.119664,...,0.000547,12.508647,1.098576,0.027195,2.347198,16.698330,2.672506,22.992043,2.639005,2.675739


In [14]:
MIC_p = test[0]
# i < j < m, MIC between row i and row j
# stored in k = m*i - i*(i+1)/2 - i - 1 + j
len(MIC_p)

import numpy as np
with open('/Users/KevinBu/Desktop/Clemente Lab/CUtIe/data/MINE/n=50,alpha=0.6.csv', 'rU') as f:
    MINE_bins, pvalue_bins = parse_minep(f, delimiter = ',', pskip = 13)

# convert MIC_p into full MIC_str array
MIC_str = np.zeros(shape=[n_var,n_var])
for i in xrange(n_var):
    for j in xrange(n_var):
        k = m*i - i*(i+1)/2 - i - 1 + j
        MIC_str[i][j] = MIC_p[k]

# compute_mine function
def compute_mine(new_var1, new_var2, pvalue_bins, mine_str, mine_bins):
    # if resulting variables do not contain enough points
    if new_var1.size < 2 or new_var2.size < 2:
        p_value = 1
        r_value = 0
    else:
        data = np.stack([new_var1, new_var2], 0)
        r_value = minepy.pstats(data, alpha=0.6, c=15, est="mic_approx")
        p_value = new_str_to_pvalues(pvalue_bins, r_value, mine_bins)
        
    return p_value, r_value

def new_str_to_pvalues(pvalue_bins, mine_str, mine_bins):
    found, midpoint = binarySearchBins(pvalue_bins, mine_str)
    if found:
        mine_p = mine_bins[midpoint][1]
    else:
        mine_p = 1
        
    return mine_p

124750

In [ ]:
def resamplek_cutie_mine(var1_index, var2_index, n_samp, samp_var1, samp_var2,
                       pvalues, threshold, resample_k, sign, fold, fold_value,
                        pvalue_bins, mine_str, mine_bins):
  
    # initialize indicators and variables
    exceeds, reverse, maxp, minr, var1, var2 = init_var_indicators(var1_index,
                                            var2_index, samp_var1, samp_var2)

    # iteratively delete k samples and recompute statistics
    combs = [list(x) for x in itertools.combinations(xrange(n_samp), resample_k)]
    for indices in combs:
        new_var1 = var1[~np.in1d(range(len(var1)),indices)]
        new_var2 = var2[~np.in1d(range(len(var2)),indices)]
       
        # remove NaNs
        new_var1, new_var2 = remove_nans(new_var1, new_var2)

        # compute new p_value and r_value
        p_value, r_value = compute_mine(new_var1, new_var2, pvalue_bins, mine_str, mine_bins)

        # update reverse, maxp, and minr
        reverse, maxp, minr = update_rev_maxp_minr(sign, r_value, p_value,
                                                   indices, reverse, maxp, minr)
        
        if fold:
            if (p_value > threshold and \
                p_value > pvalues[var1_index][var2_index] * fold_value) or \
                np.isnan(p_value):
                for i in indices:
                    exceeds[i] += 1
        elif p_value > threshold or np.isnan(p_value): 
            for i in indices:
                exceeds[i] += 1

    return reverse, exceeds, maxp, minr

In [15]:

def parse_minep(pvalue_fp, delimiter = ',', pskip = 13):
    """
    INTPUTS
    pvalue_fp: table of pvalue-MICstrength relationship provided by MINE
    delimiter: ',' MINE uses csv files by default
    pskip:     number of rows to skip in the pvalue table (various comments)

    OUTPUTS
    MINE_bins:       array where each row has [MIC_str, pvalue, stderr of pvalue]
                     (pvalue corresponds to probability of observing MIC_str as and 
                     more extreme as observed MIC_str)
    pvalues_ordered: sorted list of pvalues from greatest to least used by MINE 
                     to bin

    FUNCTION
    Parses a MINE pvalue table into bins that relate MIC-str to pvalue
    """
    # initialize lists
    MINE_bins = []
    pvalues_ordered = []
    # skip comments
    for i in xrange(pskip):
        pvalue_fp.readline()
    # parse file
    for line in pvalue_fp.readlines():
        # example line: 1.000000,0.000000256,0.000000181
        # corresonding to [MIC_str, pvalue, stderr of pvalue]
        split_line = line.rstrip().split(delimiter)
        # make sure line is valid; last line is 'xla' 
        if len(split_line) > 1:
            row = [float(x) for x in split_line]
            MINE_bins.append(row)
            pvalues_ordered.append(row[0]) # row[0] is the pvalue

    # convert list to array
    MINE_bins = np.array(MINE_bins)

    return MINE_bins, pvalues_ordered 




In [22]:
MIC_p[3]

0.32297764255298839

In [36]:
import minepy
test = minepy.pstats(df.head(), alpha=0.6, c=15, est="mic_approx")
MIC_p = test[0]
print MIC_p
n_var = 5
MIC_str = np.zeros(shape=[n_var,n_var])
for i in xrange(n_var):
    for j in xrange(n_var):
        if i == j:
            MIC_str[i][j] = 1
        elif i < j:
            k = abs(n_var*i - i*(i+1)/2 - i - 1 + j)
            MIC_str[i][j] = MIC_p[k]
            MIC_str[j][i] = MIC_p[k]
        
print MIC_str

[ 0.29257366  0.38677239  0.25509445  0.32297764  0.3120421   0.384384
  0.29087946  0.34987163  0.22419503  0.30041528]
[[ 1.          0.29257366  0.38677239  0.25509445  0.32297764]
 [ 0.29257366  1.          0.3120421   0.384384    0.29087946]
 [ 0.38677239  0.3120421   1.          0.34987163  0.22419503]
 [ 0.25509445  0.384384    0.34987163  1.          0.30041528]
 [ 0.32297764  0.29087946  0.22419503  0.30041528  1.        ]]
